# Realized Volatility Prediction


A NN using Embedding.

**References**
* NOTE: Idea code based on https://www.kaggle.com/syerwin/stock-embedding-ffnn-my-features-a5d6b0
* https://www.kaggle.com/alexioslyon/stock-embedding-ffnn-my-features


* https://www.kaggle.com/jiashenliu/introduction-to-financial-concepts-and-data
* https://www.kaggle.com/vbmokin/data-science-for-tabular-data-advanced-techniques
* Embedding layer from : https://www.kaggle.com/colinmorris/embedding-layers

**My Work**
* Code learning, refactoring and understanding.


# Import Libraries

In [ ]:
from IPython.core.display import display, HTML

import pandas as pd
import numpy as np
import glob
import os
import gc

from joblib import Parallel, delayed

from sklearn import preprocessing, model_selection
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import QuantileTransformer
from sklearn.metrics import r2_score

import matplotlib.pyplot as plt 
import seaborn as sns
import numpy.matlib


from numpy.random import seed
seed(114)
import tensorflow as tf
tf.random.set_seed(114)
from tensorflow import keras
import numpy as np
from keras import backend as K


from sklearn.cluster import KMeans


#https://bignerdranch.com/blog/implementing-swish-activation-function-in-keras/
from keras.backend import sigmoid
def swish(x, beta = 1):
    return (x * sigmoid(beta * x))
from keras.utils.generic_utils import get_custom_objects
from keras.layers import Activation
get_custom_objects().update({'swish': Activation(swish)})



# Variables & Parameters

In [ ]:
path_submissions = '/'
# data directory
data_dir = '../input/optiver-realized-volatility-prediction/'
train_filename = '../input/optiver-realized-volatility-prediction/train.csv'
test_filename = '../input/optiver-realized-volatility-prediction/test.csv'

target_name = 'target'
scores_folds = {}

# Model Parameters
learning_rate = 0.0011
num_epochs = 200 # 1000

## Functions

In [ ]:
# Function to calculate first WAP
def wap1(df):
    wap = (df['bid_price1'] * df['ask_size1'] + df['ask_price1'] * df['bid_size1']) / (df['bid_size1'] + df['ask_size1'])
    return wap

# to calculate second WAP
def wap2(df):
    wap = (df['bid_price2'] * df['ask_size2'] + df['ask_price2'] * df['bid_size2']) / (df['bid_size2'] + df['ask_size2'])
    return wap

# to calculate the log of the return
# log(xi, xi-1) = log(xi) - log(xi-1)
def log_return(series):
    return np.log(series).diff()

# to calculate the realized volatility
def realized_volatility(series):
    return np.sqrt(np.sum(series**2))

# Function to count unique elements of a series
def count_unique(series):
    return len(np.unique(series))

# Function to read our base train and test set
def read_train_test(train_filename, test_filename):
    ##TOREMOVE: _train = pd.read_csv('../input/optiver-realized-volatility-prediction/train.csv')
    ##TOREMOVE: _test = pd.read_csv('../input/optiver-realized-volatility-prediction/test.csv')
    _train = pd.read_csv(train_filename)
    _test = pd.read_csv(test_filename)
    # Create a key to merge with book and trade data
    _train['row_id'] = _train['stock_id'].astype(str) + '-' + _train['time_id'].astype(str)
    _test['row_id'] = _test['stock_id'].astype(str) + '-' + _test['time_id'].astype(str)
    print(f'The training set has {_train.shape[0]} rows')
    print(f'The test set has {_train.shape[0]} rows')
          
    ##TODO BORRAR
    #d = _train[_train['stock_id']==1]
    #return d, _test
    
    return _train, _test
          

def root_mean_squared_per_error(y_true, y_pred):
    return K.sqrt(K.mean(K.square( (y_true - y_pred)/ y_true )))

# book_preprocessor Function

In [ ]:
# to preprocess book data (for each stock id)
def book_preprocessor(file_path):
    
    print ('Init: book_preprocessor')
    
    df = pd.read_parquet(file_path)
    
    # Calculate Wap
    df['wap1'] = wap1(df)
    df['wap2'] = wap2(df)
    
    # Calculate log returns
    df['log_return1'] = df.groupby(['time_id'])['wap1'].apply(log_return)
    df['log_return2'] = df.groupby(['time_id'])['wap2'].apply(log_return)
    # Calculate wap balance
    df['wap_balance'] = abs(df['wap1'] - df['wap2'])
    # Calculate spread
    df['price_spread'] = (df['ask_price1'] - df['bid_price1']) / ((df['ask_price1'] + df['bid_price1']) / 2)
    df['price_spread2'] = (df['ask_price2'] - df['bid_price2']) / ((df['ask_price2'] + df['bid_price2']) / 2)
    df['bid_spread'] = df['bid_price1'] - df['bid_price2']
    df['ask_spread'] = df['ask_price1'] - df['ask_price2']
    df["bid_ask_spread"] = abs(df['bid_spread'] - df['ask_spread'])
    df['total_volume'] = (df['ask_size1'] + df['ask_size2']) + (df['bid_size1'] + df['bid_size2'])
    df['volume_imbalance'] = abs((df['ask_size1'] + df['ask_size2']) - (df['bid_size1'] + df['bid_size2']))
    
    # Dict for aggregations
    create_feature_dict = {
        'wap1': [np.sum, np.mean, np.std],
        'wap2': [np.sum, np.mean, np.std],
        'log_return1': [np.sum, realized_volatility, np.mean, np.std],
        'log_return2': [np.sum, realized_volatility, np.mean, np.std],
        'wap_balance': [np.sum, np.mean, np.std],
        'price_spread':[np.sum, np.mean, np.std],
        'price_spread2':[np.sum, np.mean, np.std],
        'bid_spread':[np.sum, np.mean, np.std],
        'ask_spread':[np.sum, np.mean, np.std],
        'total_volume':[np.sum, np.mean, np.std],
        'volume_imbalance':[np.sum, np.mean, np.std],
        "bid_ask_spread":[np.sum, np.mean, np.std],
    }
    
    # Function to get group stats for different windows (seconds in bucket)
    def get_stats_window(seconds_in_bucket, add_suffix = False):
        # Group by the window
        df_feature = df[df['seconds_in_bucket'] >= seconds_in_bucket].groupby(['time_id']).agg(create_feature_dict).reset_index()
        # Rename columns joining suffix
        df_feature.columns = ['_'.join(col) for col in df_feature.columns]
        # Add a suffix to differentiate windows
        if add_suffix:
            df_feature = df_feature.add_suffix('_' + str(seconds_in_bucket))
        return df_feature
    
    # Get the stats for different windows
    df_feature = get_stats_window(seconds_in_bucket = 0, add_suffix = False)
    #df_feature_450 = get_stats_window(seconds_in_bucket = 450, add_suffix = True)
    #df_feature_500 = get_stats_window(seconds_in_bucket = 500, add_suffix = True)
    df_feature_400 = get_stats_window(seconds_in_bucket = 400, add_suffix = True)
    df_feature_300 = get_stats_window(seconds_in_bucket = 300, add_suffix = True)
    df_feature_200 = get_stats_window(seconds_in_bucket = 200, add_suffix = True)
    #df_feature_150 = get_stats_window(seconds_in_bucket = 150, add_suffix = True)

    # Merge all
    #df_feature = df_feature.merge(df_feature_450, how = 'left', left_on = 'time_id_', right_on = 'time_id__450')
    df_feature = df_feature.merge(df_feature_400, how = 'left', left_on = 'time_id_', right_on = 'time_id__400')
    df_feature = df_feature.merge(df_feature_300, how = 'left', left_on = 'time_id_', right_on = 'time_id__300')
    df_feature = df_feature.merge(df_feature_200, how = 'left', left_on = 'time_id_', right_on = 'time_id__200')
    #df_feature = df_feature.merge(df_feature_150, how = 'left', left_on = 'time_id_', right_on = 'time_id__150')
    #df_feature = df_feature.merge(df_feature_100, how = 'left', left_on = 'time_id_', right_on = 'time_id__100')
    # Drop unnecesary time_ids
    #df_feature.drop(['time_id__450', 'time_id__300', 'time_id__150'], axis = 1, inplace = True)
    df_feature.drop(['time_id__400', 'time_id__300', 'time_id__200'], axis = 1, inplace = True)
    
    # Create row_id so we can merge
    stock_id = file_path.split('=')[1]
    df_feature['row_id'] = df_feature['time_id_'].apply(lambda x: f'{stock_id}-{x}')
    df_feature.drop(['time_id_'], axis = 1, inplace = True)
    
    print ('End: book_preprocessor')
    
    return df_feature

# trade_preprocessor Function

In [ ]:
# Function to preprocess trade data (for each stock id)
def trade_preprocessor(file_path):
    print ('Init: trade_preprocessor')
    df = pd.read_parquet(file_path)
    df['log_return'] = df.groupby('time_id')['price'].apply(log_return)
    
    # Dict for aggregations
    create_feature_dict = {
        'log_return':[realized_volatility],
        'seconds_in_bucket':[count_unique],
        'size':[np.sum, realized_volatility, np.mean, np.std, np.max, np.min],
        'order_count':[np.mean,np.sum,np.max],
    }
    
    # Function to get group stats for different windows (seconds in bucket)
    def get_stats_window(seconds_in_bucket, add_suffix = False):
        # Group by the window
        df_feature = df[df['seconds_in_bucket'] >= seconds_in_bucket].groupby(['time_id']).agg(create_feature_dict).reset_index()
        # Rename columns joining suffix
        df_feature.columns = ['_'.join(col) for col in df_feature.columns]
        # Add a suffix to differentiate windows
        if add_suffix:
            df_feature = df_feature.add_suffix('_' + str(seconds_in_bucket))
        return df_feature
    

    # Get the stats for different windows
    df_feature = get_stats_window(seconds_in_bucket = 0, add_suffix = False)
    #df_feature_450 = get_stats_window(seconds_in_bucket = 450, add_suffix = True)
    #df_feature_500 = get_stats_window(seconds_in_bucket = 500, add_suffix = True)
    df_feature_400 = get_stats_window(seconds_in_bucket = 400, add_suffix = True)
    df_feature_300 = get_stats_window(seconds_in_bucket = 300, add_suffix = True)
    df_feature_200 = get_stats_window(seconds_in_bucket = 200, add_suffix = True)
    #df_feature_150 = get_stats_window(seconds_in_bucket = 150, add_suffix = True)
    
    def tendency(price, vol):    
        df_diff = np.diff(price)
        val = (df_diff/price[1:])*100
        power = np.sum(val*vol[1:])
        return(power)
    
    lis = []
    for n_time_id in df['time_id'].unique():
        df_id = df[df['time_id'] == n_time_id]        
        tendencyV = tendency(df_id['price'].values, df_id['size'].values)      
        f_max = np.sum(df_id['price'].values > np.mean(df_id['price'].values))
        f_min = np.sum(df_id['price'].values < np.mean(df_id['price'].values))
        df_max =  np.sum(np.diff(df_id['price'].values) > 0)
        df_min =  np.sum(np.diff(df_id['price'].values) < 0)
        # new
        abs_diff = np.median(np.abs( df_id['price'].values - np.mean(df_id['price'].values)))        
        energy = np.mean(df_id['price'].values**2)
        iqr_p = np.percentile(df_id['price'].values,75) - np.percentile(df_id['price'].values,25)
        
        # vol vars
        
        abs_diff_v = np.median(np.abs( df_id['size'].values - np.mean(df_id['size'].values)))        
        energy_v = np.sum(df_id['size'].values**2)
        iqr_p_v = np.percentile(df_id['size'].values,75) - np.percentile(df_id['size'].values,25)
        
        lis.append({'time_id':n_time_id,'tendency':tendencyV,'f_max':f_max,'f_min':f_min,'df_max':df_max,'df_min':df_min,
                   'abs_diff':abs_diff,'energy':energy,'iqr_p':iqr_p,'abs_diff_v':abs_diff_v,'energy_v':energy_v,'iqr_p_v':iqr_p_v})
    
    df_lr = pd.DataFrame(lis)
        
   
    df_feature = df_feature.merge(df_lr, how = 'left', left_on = 'time_id_', right_on = 'time_id')
    
    # Merge all
    #df_feature = df_feature.merge(df_feature_450, how = 'left', left_on = 'time_id_', right_on = 'time_id__450')
    df_feature = df_feature.merge(df_feature_400, how = 'left', left_on = 'time_id_', right_on = 'time_id__400')
    df_feature = df_feature.merge(df_feature_300, how = 'left', left_on = 'time_id_', right_on = 'time_id__300')
    #df_feature = df_feature.merge(df_feature_150, how = 'left', left_on = 'time_id_', right_on = 'time_id__150')
    df_feature = df_feature.merge(df_feature_200, how = 'left', left_on = 'time_id_', right_on = 'time_id__200')
    # Drop unnecesary time_ids
    #df_feature.drop(['time_id__450', 'time_id__300', 'time_id__150','time_id'], axis = 1, inplace = True)
    df_feature.drop(['time_id__400', 'time_id__300', 'time_id__200','time_id'], axis = 1, inplace = True)
    
    
    df_feature = df_feature.add_prefix('trade_')
    stock_id = file_path.split('=')[1]
    df_feature['row_id'] = df_feature['trade_time_id_'].apply(lambda x:f'{stock_id}-{x}')
    df_feature.drop(['trade_time_id_'], axis = 1, inplace = True)
    
    print ('End: trade_preprocessor')
    
    return df_feature



# preprocessor Function

In [ ]:
# Function to get group stats for the stock_id and time_id
def get_time_stock(df):
    # Get realized volatility columns
    #vol_cols = ['log_return1_realized_volatility', 'log_return2_realized_volatility', 'log_return1_realized_volatility_450', 'log_return2_realized_volatility_450', 
    #            'log_return1_realized_volatility_300', 'log_return2_realized_volatility_300', 'log_return1_realized_volatility_150', 'log_return2_realized_volatility_150', 
    #           'trade_log_return_realized_volatility', 'trade_log_return_realized_volatility_450', 'trade_log_return_realized_volatility_300', 'trade_log_return_realized_volatility_150']
    vol_cols = ['log_return1_realized_volatility', 'log_return2_realized_volatility', 'log_return1_realized_volatility_400', 'log_return2_realized_volatility_400', 
                'log_return1_realized_volatility_300', 'log_return2_realized_volatility_300', 'log_return1_realized_volatility_200', 'log_return2_realized_volatility_200', 
                'trade_log_return_realized_volatility', 'trade_log_return_realized_volatility_400', 'trade_log_return_realized_volatility_300', 'trade_log_return_realized_volatility_200']
    #     vol_cols = ['log_return1_realized_volatility', 'log_return2_realized_volatility',
    #                 'log_return1_realized_volatility_600', 'log_return2_realized_volatility_600', 
    #                 'log_return1_realized_volatility_400', 'log_return2_realized_volatility_400',
    # #                 'log_return1_realized_volatility_300', 'log_return2_realized_volatility_300', 
    #                 'log_return1_realized_volatility_200', 'log_return2_realized_volatility_200',
    # #                 'log_return1_realized_volatility_100', 'log_return2_realized_volatility_100', 
    #                 'trade_log_return_realized_volatility',
    #                 'trade_log_return_realized_volatility_600', 
    #                 'trade_log_return_realized_volatility_400',
    # #                 'trade_log_return_realized_volatility_300',
    # #                 'trade_log_return_realized_volatility_100',
    #                 'trade_log_return_realized_volatility_200']

    # Group by the stock id
    df_stock_id = df.groupby(['stock_id'])[vol_cols].agg(['mean', 'std', 'max', 'min', ]).reset_index()
    # Rename columns joining suffix
    df_stock_id.columns = ['_'.join(col) for col in df_stock_id.columns]
    df_stock_id = df_stock_id.add_suffix('_' + 'stock')

    # Group by the stock id
    df_time_id = df.groupby(['time_id'])[vol_cols].agg(['mean', 'std', 'max', 'min', ]).reset_index()
    # Rename columns joining suffix
    df_time_id.columns = ['_'.join(col) for col in df_time_id.columns]
    df_time_id = df_time_id.add_suffix('_' + 'time')
    
    # Merge with original dataframe
    df = df.merge(df_stock_id, how = 'left', left_on = ['stock_id'], right_on = ['stock_id__stock'])
    df = df.merge(df_time_id, how = 'left', left_on = ['time_id'], right_on = ['time_id__time'])
    df.drop(['stock_id__stock', 'time_id__time'], axis = 1, inplace = True)
    return df
    
# Funtion to make preprocessing function in parallel (for each stock id)
def preprocessor(list_stock_ids, is_train = True):
    
    # Parrallel for loop
    def for_joblib(stock_id):
        # Train
        if is_train:
            file_path_book = data_dir + "book_train.parquet/stock_id=" + str(stock_id)
            file_path_trade = data_dir + "trade_train.parquet/stock_id=" + str(stock_id)
        # Test
        else:
            file_path_book = data_dir + "book_test.parquet/stock_id=" + str(stock_id)
            file_path_trade = data_dir + "trade_test.parquet/stock_id=" + str(stock_id)
    
        # Preprocess book and trade data and merge them
        df_tmp = pd.merge(book_preprocessor(file_path_book), trade_preprocessor(file_path_trade), on = 'row_id', how = 'left')
        
        # Return the merge dataframe
        return df_tmp
    
    # Use parallel api to call paralle for loop
    df = Parallel(n_jobs = -1, verbose = 1)(delayed(for_joblib)(stock_id) for stock_id in list_stock_ids)
    
    # Concatenate all the dataframes that return from Parallel
    df = pd.concat(df, ignore_index = True)
    return df

# Function to calculate the root mean squared percentage error
def rmspe(y_true, y_pred):
    return np.sqrt(np.mean(np.square((y_true - y_pred) / y_true)))

# Function to early stop with root mean squared percentage error
# TODO: Rreview....
def feval_rmspe(y_pred, lgb_train):
    y_true = lgb_train.get_label()
    return 'RMSPE', rmspe(y_true, y_pred), False

In [ ]:
# Funtion to make preprocessing function in parallel (for each stock id)
def prepreprocessor(list_stock_ids, is_train = True):
    
    # Parrallel for loop
    def for_joblib(stock_id):
        # Train
        if is_train:
            file_path_book = data_dir + "book_train.parquet/stock_id=" + str(stock_id)
            file_path_trade = data_dir + "trade_train.parquet/stock_id=" + str(stock_id)
        # Test
        else:
            file_path_book = data_dir + "book_test.parquet/stock_id=" + str(stock_id)
            file_path_trade = data_dir + "trade_test.parquet/stock_id=" + str(stock_id)
    
        # Preprocess book and trade data and merge them
        df_tmp = pd.merge(book_preprocessor(file_path_book), trade_preprocessor(file_path_trade), on = 'row_id', how = 'left')
        
        # Return the merge dataframe
        return df_tmp
    
    # Use parallel api to call paralle for loop
    ##df = Parallel(n_jobs = -1, verbose = 1)(delayed(for_joblib)(stock_id) for stock_id in list_stock_ids)
    
    for stock_id in list_stock_ids:
        print('stock_id: {}'.format(stock_id))
        d1 = for_joblib(stock_id)
        # Concatenate all the dataframes that return from Parallel
        ##df = pd.concat(d1, ignore_index = True)
        if stock_id==0:
            df = d1
        else:
            df.append(d1, ignore_index = True)
        
        ##if stock_id==0: break
            
    return df

# Train and Test

In [ ]:
def train_test_readisness(train_filename, test_filename):
    # Read train and test
    print ('Reading files ...')
    train, test = read_train_test(train_filename, test_filename )

    print ('Train Preprocessing ...')

    # Get unique stock ids 
    train_stock_ids = train['stock_id'].unique()

    # Preprocess them using Parallel and our single stock id functions
    ##train_ = preprocessor(train_stock_ids, is_train = True)
    train_ = prepreprocessor(train_stock_ids, is_train = True)
    train = train.merge(train_, on = ['row_id'], how = 'left')

    print ('Test Preprocessing ...')
    # Get unique stock ids 
    test_stock_ids = test['stock_id'].unique()

    # Preprocess them using Parallel and our single stock id functions
    ##test_ = preprocessor(test_stock_ids, is_train = False)
    test_ = prepreprocessor(test_stock_ids, is_train = False)
    test = test.merge(test_, on = ['row_id'], how = 'left')

    print ('Grouping stats ...')
    # Get group stats of time_id and stock_id
    train = get_time_stock(train)
    test = get_time_stock(test)

    print('Pre-Procesor done.')
    
    # replace by order sum (tau)
    train['size_tau'] = np.sqrt( 1/ train['trade_seconds_in_bucket_count_unique'] )
    test['size_tau'] = np.sqrt( 1/ test['trade_seconds_in_bucket_count_unique'] )
    train['size_tau_400'] = np.sqrt( 1/ train['trade_seconds_in_bucket_count_unique_400'] )
    test['size_tau_400'] = np.sqrt( 1/ test['trade_seconds_in_bucket_count_unique_400'] )
    train['size_tau_300'] = np.sqrt( 1/ train['trade_seconds_in_bucket_count_unique_300'] )
    test['size_tau_300'] = np.sqrt( 1/ test['trade_seconds_in_bucket_count_unique_300'] )
    train['size_tau_200'] = np.sqrt( 1/ train['trade_seconds_in_bucket_count_unique_200'] )
    test['size_tau_200'] = np.sqrt( 1/ test['trade_seconds_in_bucket_count_unique_200'] )
    
    # tau2 
    train['size_tau2'] = np.sqrt( 1/ train['trade_order_count_sum'] )
    test['size_tau2'] = np.sqrt( 1/ test['trade_order_count_sum'] )
    train['size_tau2_400'] = np.sqrt( 0.25/ train['trade_order_count_sum'] )
    test['size_tau2_400'] = np.sqrt( 0.25/ test['trade_order_count_sum'] )
    train['size_tau2_300'] = np.sqrt( 0.5/ train['trade_order_count_sum'] )
    test['size_tau2_300'] = np.sqrt( 0.5/ test['trade_order_count_sum'] )
    train['size_tau2_200'] = np.sqrt( 0.75/ train['trade_order_count_sum'] )
    test['size_tau2_200'] = np.sqrt( 0.75/ test['trade_order_count_sum'] )

    # delta tau
    train['size_tau2_d'] = train['size_tau2_400'] - train['size_tau2']
    test['size_tau2_d'] = test['size_tau2_400'] - test['size_tau2']
    
    print('Function: train_test_readisness done.')
    
    return train, test

# Training model and making predictions

In [ ]:
def kfold_knn():
    # kfold based on the knn++ algorithm

    out_train = pd.read_csv('../input/optiver-realized-volatility-prediction/train.csv')
    out_train = out_train.pivot(index='time_id', columns='stock_id', values='target')

    #out_train[out_train.isna().any(axis=1)]
    out_train = out_train.fillna(out_train.mean())
    out_train.head()

    # code to add the just the read data after first execution

    # data separation based on knn ++
    nfolds = 5 # number of folds
    index = []
    totDist = []
    values = []
    # generates a matriz with the values of 
    mat = out_train.values

    scaler = MinMaxScaler(feature_range=(-1, 1))
    mat = scaler.fit_transform(mat)

    nind = int(mat.shape[0]/nfolds) # number of individuals

    # adds index in the last column
    mat = np.c_[mat,np.arange(mat.shape[0])]

    lineNumber = np.random.choice(np.array(mat.shape[0]), size=nfolds, replace=False)
    lineNumber = np.sort(lineNumber)[::-1]

    for n in range(nfolds):
        totDist.append(np.zeros(mat.shape[0]-nfolds))

    # saves index
    for n in range(nfolds):
        values.append([lineNumber[n]])    


    s=[]
    for n in range(nfolds):
        s.append(mat[lineNumber[n],:])
        mat = np.delete(mat, obj=lineNumber[n], axis=0)

    for n in range(nind-1):    
        luck = np.random.uniform(0,1,nfolds)

        for cycle in range(nfolds):
             # saves the values of index           

            s[cycle] = np.matlib.repmat(s[cycle], mat.shape[0], 1)

            sumDist = np.sum( (mat[:,:-1] - s[cycle][:,:-1])**2 , axis=1)   
            totDist[cycle] += sumDist        

            # probabilities
            f = totDist[cycle]/np.sum(totDist[cycle]) # normalizing the totdist
            j = 0
            kn = 0
            for val in f:
                j += val        
                if (j > luck[cycle]): # the column was selected
                    break
                kn +=1
            lineNumber[cycle] = kn

            # delete line of the value added    
            for n_iter in range(nfolds):

                totDist[n_iter] = np.delete(totDist[n_iter],obj=lineNumber[cycle], axis=0)
                j= 0

            s[cycle] = mat[lineNumber[cycle],:]
            values[cycle].append(int(mat[lineNumber[cycle],-1]))
            mat = np.delete(mat, obj=lineNumber[cycle], axis=0)


    for n_mod in range(nfolds):
        values[n_mod] = out_train.index[values[n_mod]]

    
    return values

In [ ]:
def nnnn(train, test):
    
    colNames = list(train)

    colNames.remove('time_id')
    colNames.remove('target')
    colNames.remove('row_id')
    colNames.remove('stock_id')

    train.replace([np.inf, -np.inf], np.nan,inplace=True)
    test.replace([np.inf, -np.inf], np.nan,inplace=True)
    qt_train = []

    for col in colNames:
        #print(col)
        qt = QuantileTransformer(random_state=21,n_quantiles=2000, output_distribution='normal')
        train[col] = qt.fit_transform(train[[col]])
        test[col] = qt.transform(test[[col]])    
        qt_train.append(qt)
    
    # making agg features
    train_p = pd.read_csv('../input/optiver-realized-volatility-prediction/train.csv')
    train_p = train_p.pivot(index='time_id', columns='stock_id', values='target')

    corr = train_p.corr()

    ids = corr.index

    kmeans = KMeans(n_clusters=7, random_state=0).fit(corr.values)
    print('------------------------------AAAAAAAAAAAAAA--------------------------')
    print(kmeans.labels_)
    print('------------------------------AAAAAAAAAAAAAA--------------------------')

    l = []
    for n in range(7):
        l.append ( [ (x-1) for x in ( (ids+1)*(kmeans.labels_ == n)) if x > 0] )

    mat = []
    matTest = []

    n = 0
    for ind in l:
        print(ind)
        newDf = train.loc[train['stock_id'].isin(ind) ]
        newDf = newDf.groupby(['time_id']).agg(np.nanmean)
        newDf.loc[:,'stock_id'] = str(n)+'c1'
        mat.append ( newDf )

        newDf = test.loc[test['stock_id'].isin(ind) ]    
        newDf = newDf.groupby(['time_id']).agg(np.nanmean)
        newDf.loc[:,'stock_id'] = str(n)+'c1'
        matTest.append ( newDf )

        n+=1

    mat1 = pd.concat(mat).reset_index()
    mat1.drop(columns=['target'],inplace=True)

    mat2 = pd.concat(matTest).reset_index()
    
    
    
    matTest = []
    mat = []
    kmeans = []

    mat2 = pd.concat([mat2,mat1.loc[mat1.time_id==5]])

    mat1 = mat1.pivot(index='time_id', columns='stock_id')
    mat1.columns = ["_".join(x) for x in mat1.columns.ravel()]
    mat1.reset_index(inplace=True)

    mat2 = mat2.pivot(index='time_id', columns='stock_id')
    mat2.columns = ["_".join(x) for x in mat2.columns.ravel()]
    mat2.reset_index(inplace=True)
    
    nnn = ['time_id',
         'log_return1_realized_volatility_0c1',
         'log_return1_realized_volatility_1c1',     
         'log_return1_realized_volatility_3c1',
         'log_return1_realized_volatility_4c1',     
         'log_return1_realized_volatility_6c1',
         'total_volume_mean_0c1',
         'total_volume_mean_1c1', 
         'total_volume_mean_3c1',
         'total_volume_mean_4c1', 
         'total_volume_mean_6c1',
         'trade_size_mean_0c1',
         'trade_size_mean_1c1', 
         'trade_size_mean_3c1',
         'trade_size_mean_4c1', 
         'trade_size_mean_6c1',
         'trade_order_count_mean_0c1',
         'trade_order_count_mean_1c1',
         'trade_order_count_mean_3c1',
         'trade_order_count_mean_4c1',
         'trade_order_count_mean_6c1',      
         'price_spread_mean_0c1',
         'price_spread_mean_1c1',
         'price_spread_mean_3c1',
         'price_spread_mean_4c1',
         'price_spread_mean_6c1',   
         'bid_spread_mean_0c1',
         'bid_spread_mean_1c1',
         'bid_spread_mean_3c1',
         'bid_spread_mean_4c1',
         'bid_spread_mean_6c1',       
         'ask_spread_mean_0c1',
         'ask_spread_mean_1c1',
         'ask_spread_mean_3c1',
         'ask_spread_mean_4c1',
         'ask_spread_mean_6c1',   
         'volume_imbalance_mean_0c1',
         'volume_imbalance_mean_1c1',
         'volume_imbalance_mean_3c1',
         'volume_imbalance_mean_4c1',
         'volume_imbalance_mean_6c1',       
         'bid_ask_spread_mean_0c1',
         'bid_ask_spread_mean_1c1',
         'bid_ask_spread_mean_3c1',
         'bid_ask_spread_mean_4c1',
         'bid_ask_spread_mean_6c1',
         'size_tau2_0c1',
         'size_tau2_1c1',
         'size_tau2_3c1',
         'size_tau2_4c1',
         'size_tau2_6c1'] 

    train = pd.merge(train,mat1[nnn],how='left',on='time_id')
    test = pd.merge(test,mat2[nnn],how='left',on='time_id')
    mat1= []
    mat2= []
    
    return train, test

In [ ]:
def save_datasets(train, test, trained_filename, tested_filename):
    train.to_csv(trained_filename)
    test.to_csv(tested_filename)

def load_datasets(trained_filename, tested_filename):
    _train = pd.DataFrame()
    _test = pd.DataFrame()
    _train.read_csv(trained_filename)
    _test.pd.read_csv(tested_filename)
    return _train, _test

#trained_filename = 'mytrained.csv'
#tested_filename = 'mytested.csv'
#save_datasets(train, test, trained_filename, tested_filename)

# Base Model
embedding, flatenning and concatenating

In [ ]:
def define_neural_network():
    
    # INPUTS
    stock_id_input = keras.Input(shape=(1,), name='stock_id')
    num_input = keras.Input(shape=(362,), name='num_data')

    cat_data = train['stock_id']
    stock_embedding_size = 24
    hidden_units = (128,64,32)

    # Embedding, flatenning and concatenating
    stock_embedded = keras.layers.Embedding(max(cat_data)+1, stock_embedding_size, 
                                           input_length=1, name='stock_embedding')(stock_id_input)
    stock_flattened = keras.layers.Flatten()(stock_embedded)
    out = keras.layers.Concatenate()([stock_flattened, num_input])
    
    # Add one or more hidden layers
    for n_hidden in hidden_units:
        out = keras.layers.Dense(n_hidden, activation='swish')(out)
        out = keras.layers.Dropout(rate=0.02,seed=111)(out)
        
    #out = keras.layers.Concatenate()([out, num_input])

    # A single output: our predicted rating
    out = keras.layers.Dense(1, activation='linear', name='prediction')(out)
    
    model = keras.Model(
        inputs = [stock_id_input, num_input],
        outputs = out,
    )
    
    return model

# Prediction

In [ ]:
def prediction(train, test):

    model_name = 'NN'
    pred_name = 'pred_{}'.format(model_name)

    # https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html
    # K-Folds cross-validator
    # Provides train/test indices to split data in train/test sets. Split dataset into k consecutive folds.
    n_folds = 5
    kf = model_selection.KFold(n_splits=n_folds, shuffle=True, random_state=2020)
    scores_folds[model_name] = []
    counter = 1

    features_to_consider = list(train)
    features_to_consider.remove('time_id')
    features_to_consider.remove('target')
    features_to_consider.remove('row_id')
    try:
        features_to_consider.remove('pred_NN')
    except:
        pass

    # Fill na values with the mean in train & test
    train[features_to_consider] = train[features_to_consider].fillna(train[features_to_consider].mean())
    test[features_to_consider] = test[features_to_consider].fillna(train[features_to_consider].mean())

    train[pred_name] = 0
    test['target'] = 0

    # looping folds
    for n_count in range(n_folds):
        print('CV {}/{}'.format(counter, n_folds))

        ##indexes = np.arange(nfolds).astype(int)
        indexes = np.arange(n_folds).astype(int)    
        indexes = np.delete(indexes,obj=n_count, axis=0) 

        indexes = np.r_[values[indexes[0]],values[indexes[1]],values[indexes[2]],values[indexes[3]]]

        X_train = train.loc[train.time_id.isin(indexes), features_to_consider]
        y_train = train.loc[train.time_id.isin(indexes), target_name]
        X_test = train.loc[train.time_id.isin(values[n_count]), features_to_consider]
        y_test = train.loc[train.time_id.isin(values[n_count]), target_name]

        # NN   
        model = define_neural_network()

        es = tf.keras.callbacks.EarlyStopping(
                monitor='val_loss', patience=20, verbose=0,
                mode='min',restore_best_weights=True)

        plateau = tf.keras.callbacks.ReduceLROnPlateau(
                monitor='val_loss', factor=0.2, patience=7, verbose=0,
                mode='min')

        print('Compile Model')
        model.compile(
            keras.optimizers.Adam(learning_rate=learning_rate),
            loss=root_mean_squared_per_error
        )

        try:
            features_to_consider.remove('stock_id')
        except:
            pass

        num_data = X_train[features_to_consider]

        scaler = MinMaxScaler(feature_range=(-1, 1))         
        num_data = scaler.fit_transform(num_data.values)    

        cat_data = X_train['stock_id']    
        target =  y_train

        num_data_test = X_test[features_to_consider]
        num_data_test = scaler.transform(num_data_test.values)
        cat_data_test = X_test['stock_id']

        print('Fit Model')
        model.fit([cat_data, num_data], 
                  target,               
                  batch_size=2048,
                  epochs=num_epochs,
                  validation_data=([cat_data_test, num_data_test], y_test),
                  callbacks=[es, plateau],
                  validation_batch_size=len(y_test),
                  shuffle=True,
                  verbose = 1)

        print('Predict Model')
        preds = model.predict([cat_data_test, num_data_test]).reshape(1,-1)[0]

        score = round(rmspe(y_true = y_test, y_pred = preds),5)
        print('Fold {} {}: {}'.format(counter, model_name, score))
        scores_folds[model_name].append(score)

        tt =scaler.transform(test[features_to_consider].values)
        test[target_name] += model.predict([test['stock_id'], tt]).reshape(1,-1)[0].clip(0,1e10)
        #test[target_name] += model.predict([test['stock_id'], test[features_to_consider]]).reshape(1,-1)[0].clip(0,1e10)

        counter += 1
        features_to_consider.append('stock_id')

In [ ]:
def train_model(train, test, model):

    model_name = 'NN'
    pred_name = 'pred_{}'.format(model_name)

    features_to_consider = list(train)
    features_to_consider.remove('time_id')
    features_to_consider.remove('target')
    features_to_consider.remove('row_id')
    try:
        features_to_consider.remove('pred_NN')   ## No existe!!!!
    except:
        pass

    # Fill na values with the mean in train & test
    train[features_to_consider] = train[features_to_consider].fillna(train[features_to_consider].mean())
    test[features_to_consider] = test[features_to_consider].fillna(train[features_to_consider].mean())

    train[pred_name] = 0
    test['target'] = 0

    ##X_train = train.loc[train.time_id.isin(indexes), features_to_consider]
    ##y_train = train.loc[train.time_id.isin(indexes), target_name]
    ##X_test = train.loc[train.time_id.isin(values[n_count]), features_to_consider]
    ##y_test = train.loc[train.time_id.isin(values[n_count]), target_name]

    X_train = train.loc[train, features_to_consider]
    y_train = train.loc[train, target_name]
    X_test = train.loc[train, features_to_consider]
    y_test = train.loc[train, target_name]    
    

    monitor = tf.keras.callbacks.EarlyStopping(
          monitor='val_loss', patience=20, verbose=0,
          mode='min',restore_best_weights=True)

    plateau = tf.keras.callbacks.ReduceLROnPlateau(
                monitor='val_loss', factor=0.2, patience=7, verbose=0,
                mode='min')

    print('Compile Model')
    model.compile(
            keras.optimizers.Adam(learning_rate=learning_rate),
            loss=root_mean_squared_per_error
        )

    try:
        features_to_consider.remove('stock_id')
    except:
        pass

    num_data = X_train[features_to_consider]

    scaler = MinMaxScaler(feature_range=(-1, 1))         
    num_data = scaler.fit_transform(num_data.values)    

    cat_data = X_train['stock_id']    
    target =  y_train

    num_data_test = X_test[features_to_consider]
    num_data_test = scaler.transform(num_data_test.values)
    cat_data_test = X_test['stock_id']

    print('Fit Model')
    model.fit([cat_data, num_data], 
                  target,               
                  batch_size=2048,
                  epochs=num_epochs,
                  validation_data=([cat_data_test, num_data_test], y_test),
                  callbacks=[monitor, plateau],
                  validation_batch_size=len(y_test),
                  shuffle=True,
                  verbose = 1)


def predict_model(model, x, y_test, model_name):
    print('Predict Model')
    ##preds = model.predict([cat_data_test, num_data_test]).reshape(1,-1)[0]
    preds = model.predict(x).reshape(1,-1)[0]

    score = round(rmspe(y_true = y_test, y_pred = preds),5)
    print('Score {}: {}'.format(model_name, score))

    tt =scaler.transform(test[features_to_consider].values)
    test[target_name] += model.predict([test['stock_id'], tt]).reshape(1,-1)[0].clip(0,1e10)


# Submission

In [ ]:
def submission(train, test):
    test[target_name] = test[target_name]/n_folds

    score = round(rmspe(y_true = train[target_name].values, y_pred = train[pred_name].values),5)
    print('RMSPE {}: {} - Folds: {}'.format(model_name, score, scores_folds[model_name]))

    display(test[['row_id', target_name]].head(2))
    test[['row_id', target_name]].to_csv('submission.csv',index = False)
    

In [ ]:
def pre_submission(train, test):
    #test[target_name] = test[target_name]/n_folds

    score = round(rmspe(y_true = train[target_name].values, y_pred = train[pred_name].values),5)
    print('RMSPE {}: {} - Folds: {}'.format(model_name, score, scores_folds[model_name]))

    display(test[['row_id', target_name]].head(2))
    test[['row_id', target_name]].to_csv('submission.csv',index = False)

# Main

In [ ]:
%%time
##############################################################
train, test = train_test_readisness(train_filename, test_filename)
#values = kfold_knn()
#train, test = nnnn(train, test)

In [ ]:

  #  train, test = read_train_test(train_filename, test_filename )

#train

In [ ]:
#test

In [ ]:
#model = define_neural_network()

#predict_model(train, test, model)

#model.fit([cat_data, num_data], 
#                  target,               
#                  batch_size=2048,
#                  epochs=num_epochs,
#                  validation_data=([cat_data_test, num_data_test], y_test),
#                  callbacks=[es, plateau],
#                  validation_batch_size=len(y_test),
#                  shuffle=True,
#                  verbose = 1)
    
    


In [ ]:
#s = submission(train, test)
#print(np.mean(s[model_name]))